# Sección 1: KNN mediante GPU

## Carga y tratamiento de datos

In [ ]:
import os
import os.path

import cudf
import cupy as cp
import cuml

import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

%run ../utils/f_northing.py
%run ../utils/f_northing_numpy.py
%run ../utils/f_static_data.py
%run ../utils/f_utils.py

cities_to_use = ['sevilla']
#cities_to_use = ['shanghai']
#cities_to_use = cities_to_use_1()
#cities_to_use = cities_to_use_2()

columns_to_use = ['host_id', 'host_response_rate', 'host_acceptance_rate',
                  'latitude', 'longitude', 'accommodates', 'price', 'number_of_reviews', 'reviews_per_month',
                 'neighbourhood_cleansed']

In [ ]:
%%time
listings = cudf.DataFrame()

for city in cities_to_use:
    directory = '../data/' + city + '/'
    if os.path.exists(directory):
        for file in os.listdir(directory):
            if file.endswith('.csv'):
                temp_df = cudf.read_csv(directory + file, usecols = columns_to_use)
                standard_object_type(temp_df, ['host_acceptance_rate', 'neighbourhood_cleansed'])
                if listings.size == 0:
                    listings = temp_df
                else:
                    for column in listings.columns:
                        if listings[column].dtype != temp_df[column].dtype:
                            print('Found error: '+column+' type '+listings[column].dtype.name+' doesnt match '+temp_df[column].dtype.name)
                    listings = listings.append(temp_df)
                    
listings = listings.drop_duplicates().reset_index(drop=True)

type_conversion(listings, ['host_id', 'accommodates', 'number_of_reviews', 'reviews_per_month'])
column_factorize(listings, ['neighbourhood_cleansed'])

clean_format_strings(listings, ['host_response_rate', 'host_acceptance_rate'])
clean_format_price(listings, ['price'])

cupy_lat = cp.asarray(listings['latitude'])
cupy_long = cp.asarray(listings['longitude'])
n_cupy_array, e_cupy_array = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
listings['northing'] = cudf.Series(n_cupy_array).astype('float32')
listings['easting'] = cudf.Series(e_cupy_array).astype('float32')

listings.head()

## Aplicación de un algoritmo KNN para calcular distancias entre listados

In [ ]:
%%time
knn = cuml.NearestNeighbors(n_neighbors=3)
listings_cheap = listings[listings['price'] <= 50.0].reset_index(drop=True)
listings_expensive = listings[listings['price'] >= 100.0].reset_index(drop=True)
listings_cheap_cols = listings_cheap[['price', 'easting', 'northing']]
listings_expensive_cols = listings_expensive[['price', 'easting', 'northing']]
knn.fit(listings_cheap_cols)
distances, indices = knn.kneighbors(listings_expensive_cols, 3)

In [ ]:
distances

In [ ]:
indices

# Sección 2: KNN mediante CPU

## Carga y tratamiento de datos

In [ ]:
%%time
listings_cpu = pd.DataFrame()

for city in cities_to_use:
    directory = '../data/' + city + '/'
    if os.path.exists(directory):
        for file in os.listdir(directory):
            if file.endswith('.csv'):
                temp_df_cpu = pd.read_csv(directory + file, usecols = columns_to_use)
                standard_object_type(temp_df_cpu, ['host_acceptance_rate', 'neighbourhood_cleansed'])
                if listings_cpu.size == 0:
                    listings_cpu = temp_df_cpu
                else:
                    for column in listings_cpu.columns:
                        if listings_cpu[column].dtype != temp_df_cpu[column].dtype:
                            print('Found error: '+column+' type '+listings_cpu[column].dtype.name+' doesnt match '+temp_df_cpu[column].dtype.name)
                    listings_cpu = listings_cpu.append(temp_df_cpu)
                    
listings_cpu = listings_cpu.drop_duplicates().reset_index(drop=True)

type_conversion(listings_cpu, ['host_id', 'accommodates', 'number_of_reviews', 'reviews_per_month'])
column_factorize(listings_cpu, ['neighbourhood_cleansed'])

clean_format_strings(listings_cpu, ['host_response_rate', 'host_acceptance_rate'])
clean_format_price_cpu(listings_cpu, ['price'])

numpy_lat = listings_cpu['latitude'].to_numpy()
numpy_long = listings_cpu['longitude'].to_numpy()
n_numpy_array, e_numpy_array = latlong2osgbgrid_numpy(numpy_lat, numpy_long)
listings_cpu['northing'] = pd.Series(n_numpy_array).astype('float32')
listings_cpu['easting'] = pd.Series(e_numpy_array).astype('float32')
listings_cpu.head()

## Aplicación de KNN

In [ ]:
%%time
knn_cpu = NearestNeighbors(n_neighbors=3)
listings_cheap_cpu = listings_cpu[listings_cpu['price'] <= 50.0].reset_index(drop=True)
listings_expensive_cpu = listings_cpu[listings_cpu['price'] >= 100.0].reset_index(drop=True)
listings_cheap_cols_cpu = listings_cheap_cpu[['price', 'easting', 'northing']]
listings_expensive_cols_cpu = listings_expensive_cpu[['price', 'easting', 'northing']]
knn_cpu.fit(listings_cheap_cols_cpu)
distances_cpu, indices_cpu = knn_cpu.kneighbors(listings_expensive_cols_cpu, 3)

In [ ]:
distances_cpu

In [ ]:
indices_cpu

In [ ]:
%reset -f